# Movie_Recommendation
## Read the data

In [22]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [23]:
ratings_df = pd.read_csv('ml-25m/ratings.csv')
movies_df = pd.read_csv('ml-25m/movies.csv')
movies_df['movieRow'] = movies_df.index

In [24]:
ratings_df.tail()

,userId,movieId,rating,timestamp
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434
25000094,162541,63876,5.0,1240952515


In [25]:
movies_df.tail()

,movieId,title,genres,movieRow
62418,209157,We (2018),Drama,62418
62419,209159,Window of the Soul (2001),Documentary,62419
62420,209163,Bad Poems (2018),Comedy|Drama,62420
62421,209169,A Girl Thing (2001),(no genres listed),62421
62422,209171,Women of Devil's Island (1962),Action|Adventure|Drama,62422


In [26]:
ratings_df = ratings_df.iloc[:1000000]
ratings_df.tail()

,userId,movieId,rating,timestamp
999995,6747,3566,2.0,1008568732
999996,6747,3578,5.0,998488633
999997,6747,3593,2.0,997996934
999998,6747,3615,4.0,998592422
999999,6747,3623,5.0,997996257


In [27]:
## select attributes from movies_df
movies_df = movies_df[['movieRow', 'movieId', 'title']]
movies_df.tail()

,movieRow,movieId,title
62418,62418,209157,We (2018)
62419,62419,209159,Window of the Soul (2001)
62420,62420,209163,Bad Poems (2018)
62421,62421,209169,A Girl Thing (2001)
62422,62422,209171,Women of Devil's Island (1962)


In [28]:
#将rating_df中的movieId替换为行号
ratings_df = pd.merge(ratings_df, movies_df, on='movieId')
ratings_df = ratings_df[['userId', 'movieRow', 'rating']]

In [29]:
ratings_df

,userId,movieRow,rating
0,1,292,5.0
1,3,292,5.0
2,4,292,4.0
3,5,292,4.0
4,7,292,4.0
...,...,...,...
999995,6729,48888,4.0
999996,6730,14593,4.5
999997,6730,16252,5.0
999998,6730,19144,4.0


In [30]:
# 获取最大用户数目及电影数目
userNo = ratings_df['userId'].max() + 1
movieNo = ratings_df['movieRow'].max() + 1
print('max userNo: {},min movieNo: {}'.format(userNo, movieNo))

max userNo: 6748,min movieNo: 62375


In [43]:
# create a matrix: row movies, columns users
rating = np.zeros((movieNo, userNo))

flag = 0
ratings_df_length = np.shape(ratings_df)[0]
#查看矩阵ratings_df的第一维度是多少
# interrows（），对表格ratings_df进行遍历
for index, row in ratings_df.iterrows():
    #     print(index)
    #     print(row)
    # 将ratings_df表里的'movieRow'和'userId'列，填上row的‘评分’
    rating[int(row['movieRow']), int(row['userId'])] = row['rating']
rating

array([[0. , 0. , 3.5, ..., 0. , 0. , 5. ],
       [0. , 0. , 0. , ..., 4. , 0. , 3. ],
       [0. , 0. , 0. , ..., 0. , 0. , 3. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]])

In [47]:
record = rating > 0
print(record)
record = np.array(record,
                  dtype=int)  #将record的布尔型转化为0和1,0:用户没有对此电影评分；1：用户对此电影进行了评分
print(record)

[[False False  True ... False False  True]
 [False False False ...  True False  True]
 [False False False ... False False  True]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
[[0 0 1 ... 0 0 1]
 [0 0 0 ... 1 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [54]:
def normalizeRatings(rating, record):
    # m: No. of movies
    # n: No. of users
    m, n = rating.shape

    rating_mean = np.zeros((m, 1))
    # 每部电影的平均得分
    rating_norm = np.zeros((m, n))
    # 处理过的评分
    for i in range(m):
        # all indeces of records with ratings
        idx = record[i, :] != 0
        
        # calculate mean rating of each movie
        rating_mean[i] = np.mean(rating[i, idx])
        
        # normal = rating - mean
        rating_norm[i, idx] -= rating_mean[i]
        
    return rating_norm, rating_mean

In [55]:
rating_norm, rating_mean = normalizeRatings(rating, record)

E:\anaconda\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
E:\anaconda\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [56]:
rating_mean

array([[3.89355386],
       [3.31018519],
       [3.16276803],
       ...,
       [       nan],
       [       nan],
       [4.        ]])

In [57]:
rating_norm

array([[ 0.        ,  0.        , -3.89355386, ...,  0.        ,
         0.        , -3.89355386],
       [ 0.        ,  0.        ,  0.        , ..., -3.31018519,
         0.        , -3.31018519],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , -3.16276803],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [58]:
# set 0 (default) to NaN
rating_norm = np.nan_to_num(rating_norm)
rating_norm

array([[ 0.        ,  0.        , -3.89355386, ...,  0.        ,
         0.        , -3.89355386],
       [ 0.        ,  0.        ,  0.        , ..., -3.31018519,
         0.        , -3.31018519],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , -3.16276803],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [60]:
rating_mean = np.nan_to_num(rating_mean)
rating_mean

array([[3.89355386],
       [3.31018519],
       [3.16276803],
       ...,
       [0.        ],
       [0.        ],
       [4.        ]])

In [ ]:
num_features = 10
X_parameters = tf.Variable(
    tf.random_normal([movieNo, num_features], stddev=0.35))
Theta_parameters = tf.Variable(
    tf.random_normal([userNo, num_features], stddev=0.35))
#tf.Variables()初始化变量
#tf.random_normal()函数用于从服从指定正太分布的数值中取出指定个数的值，mean: 正态分布的均值。stddev: 正态分布的标准差。dtype: 输出的类型

In [ ]:
loss = 1 / 2 * tf.reduce_sum(
    ((tf.matmul(X_parameters, Theta_parameters, transpose_b=True) -
      rating_norm) * record)**2) + 1 / 2 * (tf.reduce_sum(X_parameters**2) +
                                            tf.reduce_sum(Theta_parameters**2))
#基于内容的推荐算法模型

In [ ]:
optimizer = tf.train.AdamOptimizer(1e-4)
# https://blog.csdn.net/lenbow/article/details/52218551
train = optimizer.minimize(loss)
# Optimizer.minimize对一个损失变量基本上做两件事
# 它计算相对于模型参数的损失梯度。
# 然后应用计算出的梯度来更新变量